In [1]:
import gc
import time
import numpy as np
import pandas as pd

from scipy.sparse import csr_matrix, hstack

from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split, cross_val_score
import lightgbm as lgb

In [2]:
train  = pd.read_csv('../MachineHackE-Commerce/Train.csv')
test = pd.read_csv('../MachineHackE-Commerce/Test.csv')
sub = pd.read_excel('../MachineHackE-Commerce/Sample_Submission.xlsx')


In [3]:
sub.head()

,Selling_Price
0,2921.95
1,508.47
2,802.71
3,402.79
4,751.10


In [5]:
train.head()

,Product,Product_Brand,Item_Category,Subcategory_1,Subcategory_2,Item_Rating,Date,Selling_Price
0,P-2610,B-659,bags wallets belts,bags,hand bags,4.3,2/3/2017,291.0
1,P-2453,B-3078,clothing,women s clothing,western wear,3.1,7/1/2015,897.0
2,P-6802,B-1810,home decor festive needs,showpieces,ethnic,3.5,1/12/2019,792.0
3,P-4452,B-3078,beauty and personal care,eye care,h2o plus eye care,4.0,12/12/2014,837.0
4,P-8454,B-3078,clothing,men s clothing,t shirts,4.3,12/12/2013,470.0


In [11]:
NUM_BRANDS = 790
NUM_CATEGORIES = 1001
NAME_MIN_DF = 10
MAX_FEATURES_ITEM_DESCRIPTION = 3

In [36]:
def cutting(dataset):
    pop_brand = dataset['Product'].value_counts().loc[lambda x: x.index != 'missing'].index[:NUM_BRANDS]
    dataset.loc[~dataset['brand_name'].isin(pop_brand), 'brand_name'] = 'missing'
    pop_category = dataset['category_name'].value_counts().loc[lambda x: x.index != 'missing'].index[:NUM_BRANDS]
    dataset.loc[~dataset['category_name'].isin(pop_category), 'category_name'] = 'missing'


In [ ]:
def to_categorical(dataset):
    dataset['category_name'] = dataset['category_name'].astype('category')
    dataset['brand_name'] = dataset['brand_name'].astype('category')
    dataset['item_condition_id'] = dataset['item_condition_id'].astype('category')


In [8]:
nrow_train = train.shape[0]
y = np.log1p(train["Selling_Price"])
dataset = pd.concat([train, test])
      

/home/ryan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [16]:
dataset['Item_Category'] = dataset['Item_Category'].astype('category')
dataset['Product_Brand'] = dataset['Product_Brand'].astype('category')
dataset['Subcategory_1'] = dataset['Subcategory_1'].astype('category')
dataset['Subcategory_2'] = dataset['Subcategory_2'].astype('category')

dataset['text'] = dataset.apply(lambda row: ' '.join ([str(row['Item_Category']), str(row['Subcategory_1']),str(row['Subcategory_2'])]), 
                                          axis=1)



In [19]:
cv = CountVectorizer(min_df=NAME_MIN_DF)
X_name = cv.fit_transform(dataset['Item_Category'])
    

In [20]:
cv = CountVectorizer()
X_category1 = cv.fit_transform(dataset['Subcategory_1'])
    

In [21]:
cv2 = CountVectorizer()
X_category2 = cv.fit_transform(dataset['Subcategory_2'])
    

In [22]:
tv = TfidfVectorizer(max_features=MAX_FEATURES_ITEM_DESCRIPTION,
                         ngram_range=(1, 3),
                         stop_words='english')
X_description = tv.fit_transform(dataset['text'])
    

In [23]:
lb = LabelBinarizer(sparse_output=True)
X_brand = lb.fit_transform(dataset['Product_Brand'])
    

In [30]:
def f(s):
    return s/s.max()

dataset['Product'] = dataset['Product'].str.replace(r'\D', '')

dataset['Product'] = pd.to_numeric(dataset['Product'], errors='coerce')


In [56]:
X_product = f(dataset['Product'])

In [63]:
X_prod=np.array(X_product).reshape(-1,1)

In [64]:
X_description.shape
X_category1.shape
X_category2.shape
X_name.shape
X_prod.shape

(3503, 1)

In [65]:
sparse_merge = hstack((X_description, X_brand, X_category1,X_category2, X_name , X_prod)).tocsr()
X = sparse_merge[:nrow_train]
X_test = sparse_merge[nrow_train:]
   

In [66]:
d_train = lgb.Dataset(X, label=y)
#d_valid = lgb.Dataset(valid_X, label=valid_y, max_bin=8192)
#watchlist = [d_train, d_valid]
    
params = {
        'learning_rate': 0.75,
        'application': 'regression',
        'max_depth': 3,
        'num_leaves': 100,
        'verbosity': -1,
        'metric': 'RMSE',
}


model = lgb.train(params, train_set=d_train, num_boost_round=3200, verbose_eval=100) 
preds = 0.57*model.predict(X_test)


model = Ridge(solver="sag", fit_intercept=True, random_state=205)
model.fit(X, y)
    
preds += 0.43*model.predict(X=X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_ridge.py:558: UserWarning: "sag" solver requires many iterations to fit an intercept with sparse inputs. Either set the solver to "auto" or "sparse_cg", or set a low "tol" and a high "max_iter" (especially if inputs are not standardized).
  '"sag" solver requires many iterations to fit '


In [70]:
sub['Selling_Price'] = np.expm1(preds)

sub.to_excel("submission_lgbm_ridge.xlsx", index=False)


https://mlwave.com/kaggle-ensembling-guide/